<a href="https://colab.research.google.com/github/CLmarco01/Tesina-Change-Detection/blob/main/Tesina_Change_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>